In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 武汉市
# roi = ee.FeatureCollection('users/311605001111/region/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [4]:
# 移除无效像元
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
    zenith = image.get('SOLAR_ZENITH_ANGLE')
    return image.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True))

# 图像可视化参数
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

In [5]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31') \
    .map(maskL8sr) \
    .map(water_index)
# Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

In [6]:
a = landsat_images.sort('CLOUD_COVER').first()
print(a.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [4480, 3839], 'origin': [953, 3277], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 651585, 0, -30, 2673315]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [4480, 3839], 'origin': [953, 3277], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 651585, 0, -30, 2673315]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [4480, 3839], 'origin': [953, 3277], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 651585, 0, -30, 2673315]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [4480, 3839], 'origin': [953, 3277], 'crs': 'EPSG:32649', 'crs_transform': [30, 0, 651585, 0, -30, 2673315]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, '

# 加载样本集,AWEI,threshold = 0

In [7]:
# 加载样本集,AWEI,threshold = 0
internal_awei0 = ee.FeatureCollection('users/311605001111/PearlRiverDelta_2020')
# Map.addLayer(samples, {}, "WuhanTrainingPoints_2020")
print(internal_awei0.size().getInfo())
clt_water = internal_awei0.filter(ee.Filter.eq('waterclass',1))
print(clt_water.size().getInfo())
clt_nowater = internal_awei0.filter(ee.Filter.eq('waterclass',0))
print(clt_nowater.size().getInfo())

1423
466
957


# k-mean均值过滤

In [8]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [9]:
filtered_awei0 = k_mean(internal_awei0)
print("筛选后，剩余的样本数： ",filtered_awei0.size().getInfo())

筛选后，剩余的样本数：  1407


# 训练RF训练器

In [10]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# 分类标签
label = 'waterclass'
trainedClassifier_awei0_filter = ee.Classifier.smileRandomForest(100).train(filtered_awei0,label,bands)
trainedClassifier_awei0 = ee.Classifier.smileRandomForest(100).train(internal_awei0,label,bands)

# 单景影像的验证

In [11]:
# sentinel-2
# 加载sentinel 图像
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)
# sen2 = ee.Image('COPERNICUS/S2_SR/20200209T025851_20200209T030437_T50RKU')
sen2 = ee.Image('COPERNICUS/S2_SR/20200130T025941_20200130T031015_T49QGE')
Map.addLayer(maskS2clouds(sen2.clip(roi)),{'min': 0.0,'max': 0.3,'bands': ['B8', 'B4', 'B3']},'sen2')

image2 = maskL8sr(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122044_20200218')))
Map.addLayer(image2.clip(roi),visParams,'image2')

In [12]:
dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-03-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))

print(dataset.aggregate_array('system:index').getInfo())

['20200115T030049_20200115T030323_T49QFE', '20200115T030049_20200115T030323_T49QHF', '20200130T025941_20200130T031015_T49QFE', '20200130T025941_20200130T031015_T49QFF', '20200130T025941_20200130T031015_T49QGE', '20200130T025941_20200130T031015_T49QGF', '20200130T025941_20200130T031015_T49QHE', '20200130T025941_20200130T031015_T49QHF', '20200315T025539_20200315T030729_T49QHF']


In [18]:
classify2_awei0 = image2.clip(roi).select(bands).classify(trainedClassifier_awei0).eq(1).rename('waterclass')
Map.addLayer(classify2_awei0.selfMask(),{'palette':['blue']}, 'classify2_awei0')

classify2_awei0_filter = image2.clip(roi).select(bands).classify(trainedClassifier_awei0_filter).eq(1).rename('waterclass')
Map.addLayer(classify2_awei0_filter.selfMask(),{'palette':['cyan']}, 'classify2_awei0_filter')

In [14]:
NDWI0 = image2.select('NDWI').gt(0)
Map.addLayer(NDWI0.selfMask(),{'palette':['green']}, 'ndwi0')

In [15]:
mNDWI0 = image2.select('mNDWI').gt(0)
Map.addLayer(mNDWI0.selfMask(),{'palette':['yellow']}, 'mndwi0')

In [16]:
awei0 = image2.select('AWEI').gt(0)
Map.addLayer(awei0.selfMask(),{'palette':['red']}, 'awei0')

In [ ]:
sen9 = ee.Image('COPERNICUS/S2_SR/20201225T030131_20201225T030129_T50RKU')
Map.addLayer(maskS2clouds(sen9.clip(roi)),{'min': 0.0,'max': 0.3,'bands': ['B8', 'B4', 'B3']},'sen9')

image9 = maskL8sr(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_123039_20201225')))
Map.addLayer(image9.clip(roi),visParams,'image9')

In [ ]:
classify9_awei0_filter = image9.clip(roi).select(bands).classify(trainedClassifier_awei0_filter).eq(1).rename('waterclass')
Map.addLayer(classify9_awei0_filter.selfMask(),{'palette':['cyan']}, 'classify9_awei0_filter')

classify9_awei0 = image9.clip(roi).select(bands).classify(trainedClassifier_awei0).eq(1).rename('waterclass')
Map.addLayer(classify9_awei0.selfMask(),{'palette':['blue']}, 'classify9_awei0')

In [ ]:
city = ee.FeatureCollection('users/311605001111/ChinaCity')
Map.addLayer(city, {}, "ChinaCity")

Maryland = ee.Image('users/311605001111/Maryland_nationwide_2018').clipToCollection(city)
Map.addLayer(Maryland, {'palette':['white','blue'],'min':0,'max':100}, "Maryland2018")

from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [72, 17, 136, 55.5] 
vis = {'min':0, 'max':100}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(Maryland, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[10,10], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(Maryland)', fontsize=20)

In [ ]:
basemap = ee.Image.constant(0).clipToCollection(city).rename('waterClass')
jrc = ee.Image('JRC/GSW1_3/YearlyHistory/2018').remap([0,1,2,3],[0,0,1,2]).rename('waterClass').clipToCollection(city)
JRC = ee.ImageCollection([jrc,basemap]).sum()
Map.addLayer(JRC, {'palette':['ffffff', 'ffbbbb', '0000ff'],'min':0,'max':2}, "Jrc")

from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [72, 17, 136, 55.5] 
vis = {'palette':['ffffff', 'ffbbbb', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(JRC, region=region, vis_params=vis)
cartoee.add_colorbar(ax, vis,loc="right",label="water frequency", orientation="vertical",discrete=True)
cartoee.add_gridlines(ax, interval=[10,10], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(JRC)', fontsize=20)